# 2-11. 프로젝트 (1): 손글씨를 분류해봅시다
 
digits 데이터로 5가지 모델을 학습시키고 각각 예측을 진행했다. 각 모델의 예측 결과와 실제 라벨을 비교하면, SVM 모델로 예측한 결과의 정확도가 0.99로 가장 높았다. 정확도 뿐만 아니라 모든 성능 지표에서 전반적으로 SVM이 높은 수준을 기록한 만큼, 이 프로젝트에는 SVM 모델을 사용하는 게 적합해 보인다.
  
이 데이터는 라벨이 0~9로 불균형한 데이터라고 보기 어려우므로 정확도를 참고해도 무리가 없다고 판단했다. LMS에서 예시로 든 스팸 메일이나 암 진단같이 FN이나 FP의 중요도를 따질 필요가 없으므로, 정확도를 지표로 삼아도 문제가 없을 것으로 보인다. 

## 필요한 모듈 import 하기

In [1]:
from sklearn.datasets import load_digits # (1) digits
from sklearn.datasets import load_wine # (2) wine
from sklearn.datasets import load_breast_cancer # (3) breast_cancer

import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier # Decision Tree
from sklearn.ensemble import RandomForestClassifier # Random Forest
from sklearn import svm # SVM
from sklearn.linear_model import SGDClassifier # SGD Classifier
from sklearn.linear_model import LogisticRegression # Logistic Regression

from sklearn.metrics import classification_report

## 데이터 준비

In [2]:
digits = load_digits()

## 데이터 이해하기

In [3]:
digits_data = digits.data
digits_labels = digits.target
# digits_target_names = digits.target_names

# 데이터 describe 해보기
digits_df = pd.DataFrame(data=digits_data, columns=digits.feature_names)
digits_df['label'] = digits_labels
digits_df

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7,label
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792,0.0,0.0,4.0,10.0,13.0,6.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,2.0,14.0,15.0,9.0,0.0,0.0,9
1793,0.0,0.0,6.0,16.0,13.0,11.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,16.0,14.0,6.0,0.0,0.0,0
1794,0.0,0.0,1.0,11.0,15.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,9.0,13.0,6.0,0.0,0.0,8
1795,0.0,0.0,2.0,10.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,12.0,16.0,12.0,0.0,0.0,9


In [4]:
print(digits.DESCR)

.. _digits_dataset:

Optical recognition of handwritten digits dataset
--------------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each blo

## train, test 데이터 분리

In [5]:
X_train, X_test, y_train, y_test = train_test_split(digits_data, 
                                                    digits_labels, 
                                                    test_size=0.2, 
                                                    random_state=25)

## 다양한 모델로 학습시켜보기

### Decision Tree

In [6]:
# Decision Tree

digits_decision_tree = DecisionTreeClassifier(random_state=32) # 모델 저장
# print(digits_decision_tree._estimator_type)

digits_decision_tree.fit(X_train, y_train) # 모델 학습

y_pred = digits_decision_tree.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       0.97      0.94      0.96        36
           1       0.67      0.86      0.76        36
           2       0.79      0.79      0.79        33
           3       0.91      0.78      0.84        40
           4       0.76      0.85      0.80        40
           5       0.94      0.86      0.90        37
           6       0.94      0.86      0.90        35
           7       0.94      0.76      0.84        38
           8       0.74      0.81      0.77        31
           9       0.81      0.85      0.83        34

    accuracy                           0.84       360
   macro avg       0.85      0.84      0.84       360
weighted avg       0.85      0.84      0.84       360



### Random Forest

In [7]:
# Random Forest

digits_random_forest = RandomForestClassifier(random_state=32) # 모델 저장
# print(digits_random_forest._estimator_type)

digits_random_forest.fit(X_train, y_train) # 모델 학습

y_pred = digits_random_forest.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       0.97      1.00      0.99        36
           2       1.00      1.00      1.00        33
           3       1.00      0.95      0.97        40
           4       0.98      1.00      0.99        40
           5       0.95      0.97      0.96        37
           6       1.00      0.94      0.97        35
           7       1.00      0.95      0.97        38
           8       0.88      0.94      0.91        31
           9       0.91      0.94      0.93        34

    accuracy                           0.97       360
   macro avg       0.97      0.97      0.97       360
weighted avg       0.97      0.97      0.97       360



### Support Vector Machine (SVM)

In [8]:
# SVM

digits_svm = svm.SVC()
# print(digits_svm._estimator_type)

digits_svm.fit(X_train, y_train) # 모델 학습

y_pred = digits_svm.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00        36
           2       0.97      1.00      0.99        33
           3       1.00      0.97      0.99        40
           4       1.00      1.00      1.00        40
           5       1.00      0.97      0.99        37
           6       1.00      1.00      1.00        35
           7       1.00      0.97      0.99        38
           8       0.94      1.00      0.97        31
           9       0.94      0.94      0.94        34

    accuracy                           0.99       360
   macro avg       0.99      0.99      0.99       360
weighted avg       0.99      0.99      0.99       360



### Stochastic Gradient Descent (SGD) Classifier

In [9]:
# SGD Classifier

digits_sgd = SGDClassifier()
# print(digits_sgd._estimator_type)

digits_sgd.fit(X_train, y_train)

y_pred = digits_sgd.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       0.83      0.97      0.90        36
           2       0.97      0.97      0.97        33
           3       1.00      0.88      0.93        40
           4       1.00      0.97      0.99        40
           5       0.97      0.95      0.96        37
           6       1.00      0.94      0.97        35
           7       0.97      0.87      0.92        38
           8       0.83      0.94      0.88        31
           9       0.86      0.94      0.90        34

    accuracy                           0.94       360
   macro avg       0.94      0.94      0.94       360
weighted avg       0.95      0.94      0.94       360



### Logistic Regression

In [10]:
# Logistic Regression

digits_logistic_regression = LogisticRegression(max_iter=10000)
# print(digits_logistic_regression._estimator_type)

digits_logistic_regression.fit(X_train, y_train)

y_pred = digits_logistic_regression.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       0.95      0.97      0.96        36
           2       1.00      0.97      0.98        33
           3       1.00      0.97      0.99        40
           4       0.97      0.97      0.97        40
           5       0.95      0.95      0.95        37
           6       1.00      0.94      0.97        35
           7       1.00      0.89      0.94        38
           8       0.88      0.94      0.91        31
           9       0.85      0.97      0.90        34

    accuracy                           0.96       360
   macro avg       0.96      0.96      0.96       360
weighted avg       0.96      0.96      0.96       360



# 2-12. 프로젝트 (2): 와인을 분류해봅시다

wine 데이터로 5가지 모델을 학습시키고 각각 예측을 진행했다. wine 데이터의 경우에는 Random Forest 모델의 정확도가 1.00으로 가장 높았다. Logistic Regression 모델 역시 정확도가 1.00이었다. Decision Tree 모델이 0.92로 정확도가 유일하게 0.9 이상이었다. SVM은 정확도가 0.7 수준으로 높지 않았다. SGD Classifier는 작은 데이터에서 학습이 잘 되지 않는 모습을 보여 판단에서 제외했다.  

Random Forest 모델의 정확도가 가장 높긴 했지만, 찾아보니 Random Forest 모델은 트리가 적으면 random_state 값에 따라 전혀 다른 모델이 만들어지기도 한다고 한다. wine 데이터는 총 178개로 많지 않은 만큼, 1.00이라는 비현실적인 값이 나온 Random Forest 모델은 적합하지 않은 것으로 보인다. 마찬가지로 정확도가 1.00인 Logistic Regression 모델은 찾아보니 이진 분류에 최적화된 모델이라고 한다. wine 데이터는 클래스가 0, 1, 2 3가지이므로 Logistic Regression 모델도 적합하지 않아 보인다. 남은 모델 중 정확도가 가장 높고 유일하게 0.9를 넘기도 한 Decision Tree를 사용하는 게 적합해 보인다.

## 필요한 모듈 import 하기

## 데이터 준비

In [11]:
wine = load_wine()

## 데이터 이해하기

In [12]:
wine_data = wine.data
wine_labels = wine.target
# wine_target_names = wine.target_names

# 데이터 describe 해보기
wine_df = pd.DataFrame(data=wine_data, columns=wine.feature_names)
wine_df['label'] = wine_labels
wine_df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,label
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


In [13]:
print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

## train, test 데이터 분리

In [14]:
X_train, X_test, y_train, y_test = train_test_split(wine_data, 
                                                    wine_labels, 
                                                    test_size=0.2, 
                                                    random_state=25)

## 다양한 모델로 학습시켜보기

### Decision Tree

In [15]:
# Decision Tree

wine_decision_tree = DecisionTreeClassifier(random_state=32) # 모델 저장
# print(wine_decision_tree._estimator_type)

wine_decision_tree.fit(X_train, y_train) # 모델 학습

y_pred = wine_decision_tree.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       0.91      0.91      0.91        11
           1       0.94      0.89      0.91        18
           2       0.88      1.00      0.93         7

    accuracy                           0.92        36
   macro avg       0.91      0.93      0.92        36
weighted avg       0.92      0.92      0.92        36



### Random Forest

In [16]:
# Random Forest

wine_random_forest = RandomForestClassifier(random_state=32) # 모델 저장
# print(wine_random_forest._estimator_type)

wine_random_forest.fit(X_train, y_train) # 모델 학습

y_pred = wine_random_forest.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        18
           2       1.00      1.00      1.00         7

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



### Support Vector Machine (SVM)

In [17]:
# SVM

wine_svm = svm.SVC()
# print(wine_svm._estimator_type)

wine_svm.fit(X_train, y_train) # 모델 학습

y_pred = wine_svm.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       0.89      0.73      0.80        11
           1       0.82      0.78      0.80        18
           2       0.40      0.57      0.47         7

    accuracy                           0.72        36
   macro avg       0.70      0.69      0.69        36
weighted avg       0.76      0.72      0.74        36



### Stochastic Gradient Descent (SGD) Classifier

In [18]:
# SGD Classifier

wine_sgd = SGDClassifier()
# print(wine_sgd._estimator_type)

wine_sgd.fit(X_train, y_train)

y_pred = wine_sgd.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       0.75      0.82      0.78        11
           1       0.71      0.94      0.81        18
           2       0.00      0.00      0.00         7

    accuracy                           0.72        36
   macro avg       0.49      0.59      0.53        36
weighted avg       0.58      0.72      0.64        36



/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Logistic Regression

In [19]:
# Logistic Regression

wine_logistic_regression = LogisticRegression(max_iter=10000)
# print(wine_logistic_regression._estimator_type)

wine_logistic_regression.fit(X_train, y_train)

y_pred = wine_logistic_regression.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        18
           2       1.00      1.00      1.00         7

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



# 2-13. 프로젝트 (3): 유방암 여부를 진단해봅시다
   
breast_cancer 데이터로 5가지 모델을 학습시키고 각각 예측을 진행했다. 암 진단의 경우 LMS에 설명되어 있던 대로 Recall 값이 중요하다. Recall은 예측이 제대로 된 것들 중 참인 것의 비율(TP / FN + TP)이다. 암 진단의 경우 암인데 암이 아니라고 판정하면 치명적인 상황에 이를 수 있기 때문이다.

5개 모델의 학습 결과와 실제 라벨을 비교했을 때, SVM 모델 예측 결과의 최종 Recall 값이 0.99로 가장 높았다. 이 프로젝트에는 SVM 모델을 사용하는 게 적합해 보인다.

## 필요한 모듈 import 하기

## 데이터 준비

In [20]:
breast_cancer = load_breast_cancer()

## 데이터 이해하기

In [21]:
breast_cancer_data = breast_cancer.data
breast_cancer_labels = breast_cancer.target
# breast_cancer_target_names = breast_cancer.target_names

# 데이터 describe 해보기
breast_cancer_df = pd.DataFrame(data=breast_cancer_data, columns=breast_cancer.feature_names)
breast_cancer_df['label'] = breast_cancer_labels
breast_cancer_df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,label
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [22]:
print(breast_cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

## train, test 데이터 분리

In [23]:
X_train, X_test, y_train, y_test = train_test_split(breast_cancer_data, 
                                                    breast_cancer_labels, 
                                                    test_size=0.2, 
                                                    random_state=25)

## 다양한 모델로 학습시켜보기

### Decision Tree

In [24]:
# Decision Tree

breast_cancer_decision_tree = DecisionTreeClassifier(random_state=32) # 모델 저장
# print(breast_cancer_decision_tree._estimator_type)

breast_cancer_decision_tree.fit(X_train, y_train) # 모델 학습

y_pred = breast_cancer_decision_tree.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       0.89      0.87      0.88        39
           1       0.93      0.95      0.94        75

    accuracy                           0.92       114
   macro avg       0.91      0.91      0.91       114
weighted avg       0.92      0.92      0.92       114



### Random Forest

In [25]:
# Random Forest

breast_cancer_random_forest = RandomForestClassifier(random_state=32) # 모델 저장
# print(breast_cancer_random_forest._estimator_type)

breast_cancer_random_forest.fit(X_train, y_train) # 모델 학습

y_pred = breast_cancer_random_forest.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       0.92      0.90      0.91        39
           1       0.95      0.96      0.95        75

    accuracy                           0.94       114
   macro avg       0.93      0.93      0.93       114
weighted avg       0.94      0.94      0.94       114



### Support Vector Machine (SVM)

In [26]:
# SVM

breast_cancer_svm = svm.SVC()
# print(breast_cancer_svm._estimator_type)

breast_cancer_svm.fit(X_train, y_train) # 모델 학습

y_pred = breast_cancer_svm.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       0.97      0.77      0.86        39
           1       0.89      0.99      0.94        75

    accuracy                           0.91       114
   macro avg       0.93      0.88      0.90       114
weighted avg       0.92      0.91      0.91       114



### Stochastic Gradient Descent (SGD) Classifier

In [27]:
# SGD Classifier

breast_cancer_sgd = SGDClassifier()
# print(breast_cancer_sgd._estimator_type)

breast_cancer_sgd.fit(X_train, y_train)

y_pred = breast_cancer_sgd.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       0.97      0.72      0.82        39
           1       0.87      0.99      0.93        75

    accuracy                           0.89       114
   macro avg       0.92      0.85      0.87       114
weighted avg       0.90      0.89      0.89       114



### Logistic Regression

In [28]:
# Logistic Regression

breast_cancer_logistic_regression = LogisticRegression(max_iter=10000)
# print(breast_cancer_logistic_regression._estimator_type)

breast_cancer_logistic_regression.fit(X_train, y_train)

y_pred = breast_cancer_logistic_regression.predict(X_test) # 예측
print(classification_report(y_test, y_pred)) # 예측 결과 비교

              precision    recall  f1-score   support

           0       0.90      0.90      0.90        39
           1       0.95      0.95      0.95        75

    accuracy                           0.93       114
   macro avg       0.92      0.92      0.92       114
weighted avg       0.93      0.93      0.93       114

